In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as Soup
from selenium import webdriver
import ast
import os
import time
import itertools as it

In [ ]:
#Run GetWikiTables.py and call this block
with open('links.txt') as f:
    links = f.read()
    links = links.split('\n')
getTablesFromLinks(links)

In [ ]:
# # Gets HTML Data for extracting indices and columns of Table 10.01
# #Run with the URL Variable

# URL = '''https://www.eia.gov/totalenergy/data/browser/?tbl=T10.01#/?f=M&start=197301&end=201901&charted='''

# def getEIATableWebSource(tableid=None):
#     '''
#     Fetches EIA Table Source and saves it in 
#     '''
    
#     chromePath = r'chromedriver.exe'
#     driver = webdriver.Chrome(chromePath)
#     driver.get(url)
#     time.sleep(10)
#     soup = Soup(driver.page_source)
#     with open('eia_html.txt' ,"w") as f:
#         f.write(str(soup.encode("utf-8")))
#         f.close()
#     return 'You can view the saved data in eia_html.txt'

In [ ]:
# #Extract indices and columns from html source above

# def extractIndicesNColNames(filename):
#     '''
    
#     '''
    
#     with open(filename) as f:
#         soup = Soup(f, "html.parser")
#         f.close()

#     myindices = soup.findAll("span", {"class": "description_text_wrapper"})
#     mycolumnnames = soup.findAll("span", {"class": "slick-column-name"})

#     indices = [(i.contents)[0] for i in myindices]
#     columns = [(i.contents) for i in mycolumnnames]
#     months = np.asarray([i[0] for i in list(it.filterfalse(lambda x: str(x) in ['[]', "[\' \']"], columns))])
#     return (months, indices)

In [ ]:
def getColumnsAndIndices(chromePath, tablenames, link1, link2):
    '''
    Returns the columns and indices for the EIA data as a dictionary.
    The keys of the dictionary are tablenames, the values are a list of
    ((list of columns) and (list of indices))
    
    Input:
    chromePath: str
    tablenames: list(str)
    link1: str
    link2: str
    
    Output: dict(str: list(list(str), list(str)))
    '''
    
    assert isinstance(chromePath, str)
    assert isinstance(tablenames, list)
    assert isinstance(link1, str)
    assert isinstance(link2, str)
    
    driver = webdriver.Chrome(chromePath)
    monthcoldict = {}
    for tablename in tablenames:
        url = 'https://www.eia.gov/totalenergy/data/browser/?tbl=T' + tablename + '#/?f=M'
        driver.get(url)
        time.sleep(5)
        soup = Soup(driver.page_source)
        myindices = soup.findAll("span", {"class": "description_text_wrapper"})
        mycolumnnames = soup.findAll("span", {"class": "slick-column-name"})
        
        indices = [(i.contents)[0].strip() for i in myindices]
        columns = [(i.contents) for i in mycolumnnames]
        
        months = np.asarray([i[0] for i in list(it.filterfalse(lambda x: str(x) in ['[]', "[\' \']"], columns))])
        monthcoldict[tablename] = [months, indices]
    driver.quit()
    return monthcoldict

In [ ]:
#Save EIA Data year wise in a folder
#Above links are used to open certain tables from the tablenames, according to the months and indices extracted above

def getTableDataFromEIA(link1=None, link2=None, inpath=None, driver=None, year1=None, year2=None):
    '''
    Returns tabledata from EIA website by generating the appropriate link.
    Saves the returned data in an aptly names file.
    
    Input:
    link1: str
    link2: str
    inpath: str
    driver: selenium.webdriver object
    year1: int
    year2: int
    '''
    assert isinstance(link1, str)
    assert isinstance(link2, str)
    assert isinstance(inpath, str)
    assert isinstance(driver, webdriver)
    assert isinstance(year1, int)
    assert isinstance(year2, int)
    
    for year in range(year1, year2+1):
            i = str(year) + str("{:02d}".format(month))
            j = str(year) + str("{:02d}".format(month+11))
            link3 = 'start='+i+'&end='+j
            if year == 2019:
                j=i
            driver.get(link1+link3+link2)
            time.sleep(5)
            html = driver.page_source
            soup = Soup(html)
            elements = soup.findAll('div', {'class' : ['slick-cell l3 r3', 'slick-cell l4 r4', 'slick-cell l5 r5', 'slick-cell l6 r6', 'slick-cell l7 r7', 'slick-cell l8 r8', 'slick-cell l9 r9', 'slick-cell l10 r10', 'slick-cell l11 r11', 'slick-cell l12 r12', 'slick-cell l13 r13', 'slick-cell l14 r14']})                           
            collist = []
            for element in elements:
                collist.append(element.contents)

            path = inpath + '/renewable'+i+'-'+j+'.txt'
            with open(path, 'w') as f:
                print('Saving data for ' + i + ' to ' + j + ' to path ' + path)
                f.write(str(collist))

In [ ]:
def getTablesFromEIA(link1, link2, link3, tablenames, chromePath, year1, year2):
    '''
    Fetches EIA Tables By Generating Apt Links.
    Uses getTableDataFromEIA as a helper function.
    Saves the data in aptly named txt files.
    
    Input:
    link1: str
    link2: str
    link3: str
    tablenames: list(str)
    chromePath: str
    year1: int
    year2: int
    '''
    assert isinstance(link1, str)
    assert isinstance(link2, str)
    assert isinstance(link3, str)
    assert isinstance(tablenames, list)
    assert all(isinstance(i, str) for i in tablenames)
    assert isinstance(chromepath, str)
    assert isinstance(year1, int)
    assert isinstance(year2, int)
    
    outtables = []
    try:
        os.mkdir('eiadata')
    except FileExistsError:
        pass
    driver = webdriver.Chrome(chromePath)
    for tablename in tablenames:
        foldername = 'eiadata/'+tablename
        try:
            os.mkdir(foldername)
        except FileExistsError:
            pass
        getTableDataFromEIA(link1+tablename+link2, link3, foldername, driver, year1, year2)
    driver.quit()

In [ ]:
def getDataFromSavedTables(tablenames=None, monthcoldict=None, year1=None, year2=None):
    '''
    Save data from saved txt files into list of pandas objects.
    
    Input:
    tablenames: list(str)
    monthcoldict: dict(str: list(list(str), list(str)))
    year1: int
    year2: int
    
    Output: list(list(pandas))
    '''
    
    assert isinstance(tablenames, list)
    assert all(isinstance(i, str) for i in tablenames)
    assert isinstance(monthcoldict, dict)
    assert all(isinstance(i, str) for i in monthcoldict.keys())
    assert all(isinstance(i, list) for i in monthcoldict.values())
    assert all(isinstance(i, list) for i[0] in j for j in monthcoldict.values())
    assert all(isinstance(i, list) for i[1] in j for j in monthcoldict.values())
    assert all(isinstance(i, list) for i in j[0] for j in k for k in monthcoldict.values())
    assert all(isinstance(i, list) for i in j[1] for j in k for k in monthcoldict.values())
    assert isinstance(year1, int)
    assert isinstance(year2, int)
    
    outtables = []
    for tablename in tablenames:
        table1 = []
        months = monthcoldict[tablename][0]
        indices = monthcoldict[tablename][1]
        for num, year in enumerate(range(year1, year2+1)):
            i = str(year) + str("{:02d}".format(month))
            j = str(year) + str("{:02d}".format(month+11))
            if year == 2019:
                j = i
            path = 'eiadata/'+ tablename + '/renewable'+i+'-'+j+'.txt'
            with open(path, 'r') as f:
                lista = f.read()
                f.close()
            listb = ast.literal_eval(lista)
            
            listc = []
            
            for i in listb:
                if (len(i) > 0) and (i != ['NA']) and (i != ['-']) and (i != ['NM']): # Neutralizing empty data
                    listc.append(i[0])
                else:
                    listc.append(0)
            
#             try:
            listc = np.array(listc).astype(float)
            listc = listc.reshape(len(indices), -1)
#             except Exception as e:
#             print(e)
#             print(tablename, year, listc)
            
            try:
                table1.append(pd.DataFrame(data=listc, columns=months[(12*num):(12*(num+1))], index=indices))
            except ValueError as e:
                if(listc.shape[1] > 1):
                    listc = listc[::, 0:1].reshape(1, -1).flatten()
                table1.append(pd.DataFrame(data=listc, columns=months[(12*num):(12*(num+1))], index=indices))
        outtables.append(table1)
    return outtables
        

In [ ]:
# Saving Data to CSV Files
def saveDataToCSV(outtables=None, tablenames=None):
    '''
    Save outtables data to CSV Files.
    
    Input:
    outtables: list(list(pandas))
    tablenames: str
    '''
    
    assert isinstance(outtables, list)
    assert all((isinstance(i, list) and all(isinstance(j, pd.DataFrame) for j in i)) for i in outtables)
    
    for num, tables in enumerate(outtables):
        for table in tables:
            column = str(table.columns[0]).split()[0] + '-' + str(table.columns[-1])
            path = 'eiadata/' + tablenames[num] + '/' + column + '.csv'
            print('Saving to ' + path)
            try:
                table.to_csv(path)
            except:
                print('No folder names \'eiadata/\'!')
            print('Successfully saved to ' + path + '!')            

In [ ]:
#Test above written codes

tablenames = ['10.01', '10.02A', '10.02B', '10.02C', '10.05', '10.06']
chromePath = r'chromedriver.exe' #Link to Selenium's Automated Chrome Driver
link1 = '''https://www.eia.gov/totalenergy/data/browser/?tbl=T'''
link2 = '''#/?f=M&'''
link3 = '''&charted='''

#Gets Tablenames
monthcoldict = getColumnsAndIndices(chromePath, tablenames, link1, link2)

year1 = 1973
year2 = 2019

#link1, link2 and link3 construct the final link to be parsed into Selenium
getTablesFromEIA(link1, link2, link3, tablenames, chromePath, year1, year2)

# Extract Data From Above Saved Tables
outtables = []
outtables = getDataFromSavedTables(tablenames, monthcoldict, year1, year2)

#Saves outtables data to csv files
saveDataToCSV(outtables, tablenames)